In [1]:
from pandas import DataFrame, Series
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
# read in the main training csv
prevapp = pd.read_csv('previous_application.csv', na_values=['','XNA','XAP'])

C:\Users\catym\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (15,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
prevapp.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT',
       'AMT_GOODS_PRICE', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'FLAG_LAST_APPL_PER_CONTRACT',
       'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT',
       'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED',
       'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'DAYS_DECISION',
       'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE',
       'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO',
       'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'SELLERPLACE_AREA',
       'NAME_SELLER_INDUSTRY', 'CNT_PAYMENT', 'NAME_YIELD_GROUP',
       'PRODUCT_COMBINATION', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE',
       'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION',
       'NFLAG_INSURED_ON_APPROVAL'], dtype=object)

In [44]:
import pandas as pd 
prevapp = pd.read_csv('prevapp_train.csv', na_values=['','XNA','XAP'])

## CNT_Payment - Term of Loans 

In [45]:
CNT = prevapp[['SK_ID_PREV' , 'SK_ID_CURR', 'NAME_CONTRACT_STATUS','CNT_PAYMENT']]
CNT.head(5)
CNT.fillna(CNT['CNT_PAYMENT'].median(),inplace=True)
a2=len(CNT[CNT['CNT_PAYMENT'].isna()])
b2 = len(CNT['CNT_PAYMENT'])
print("The percentage of missing values for middle interest yield is")
float(a2/b2)


The percentage of missing values for middle interest yield is


0.0

In [46]:
CNT_gp=CNT.groupby('SK_ID_CURR')[['CNT_PAYMENT']].agg(['min' , 'max', 'mean'])
CNT_gb_df = pd.DataFrame(CNT_gp)
CNT_gb_df
CNT_gb_df.columns = CNT_gb_df.columns.get_level_values(1)
prevapp_terms = CNT_gb_df.reset_index()
prevapp_terms.head(5)


,SK_ID_CURR,min,max,mean
0,100002,24.0,24.0,24.0
1,100003,6.0,12.0,9.0
2,100004,4.0,4.0,4.0
3,100006,0.0,48.0,18.0
4,100007,18.0,48.0,30.0


In [47]:
prevapp_terms.rename(columns={"min": "min_term", "max":"max_term",\
  "mean": "mean_term"                            },
                      inplace=True)
prevapp_terms.head(5)

,SK_ID_CURR,min_term,max_term,mean_term
0,100002,24.0,24.0,24.0
1,100003,6.0,12.0,9.0
2,100004,4.0,4.0,4.0
3,100006,0.0,48.0,18.0
4,100007,18.0,48.0,30.0


## Days_Termination
### Relative to application date of current application when was the expected termination of the previous application

In [48]:
days_terminat = prevapp[['SK_ID_PREV' , 'SK_ID_CURR', 'DAYS_TERMINATION']]

In [49]:
day_terminat= days_terminat.groupby('SK_ID_CURR')['DAYS_TERMINATION'].max()
prevapp_termination_days= pd.DataFrame(day_terminat).reset_index()
prevapp_termination_days.head(7)
prevapp_termination_days.fillna(prevapp_termination_days['DAYS_TERMINATION'].mean(),inplace=True)
prevapp_termination_days['DAYS_TERMINATION'].value_counts()/prevapp_termination_days['DAYS_TERMINATION'].value_counts().sum()
prevapp_termination_days['DAYS_TERMINATION'].replace(365243,0,inplace= True)
prevapp_termination_days.head(5)
a2=len(prevapp_termination_days[prevapp_termination_days['DAYS_TERMINATION'].isna()])
b2 = len(prevapp_termination_days['DAYS_TERMINATION'])
print("The percentage of missing values for middle interest yield is")
float(a2/b2)

The percentage of missing values for middle interest yield is


0.0

## Combining prevapp_terms, prevapp_interest_yield, prevapp_termination_days

In [64]:
merge0=prevapp_terms.merge(prevapp_termination_days, on = "SK_ID_CURR")
print(len(merge0))
merge0.head(5)
prevapp_final_train = pd.read_csv('prevapp_final_train.csv', na_values=['','XNA','XAP'])
merge0.rename(columns={"SK_ID_CURR": "ID_CURR"}, inplace=True)
prevapp_finally_train = prevapp_final_train.merge(merge0, on='ID_CURR')
print(len(prevapp_finally_train))
prevapp_finally_train.head(5)
prevapp_finally_train.to_csv('prevapp_finally_train.csv',index=False)

262331
262331


Categorical variables:
SK_ID_PREV --------------------------
SK_ID_CURR --------------------------
NAME_CONTRACT_TYPE
WEEKDAY_APPR_PROCESS_START
HOUR_APPR_PROCESS_START -------------
FLAG_LAST_APPL_PER_CONTRACT
NFLAG_LAST_APPL_IN_DAY --------------
NAME_CASH_LOAN_PURPOSE
NAME_CONTRACT_STATUS
NAME_PAYMENT_TYPE
CODE_REJECT_REASON
NAME_TYPE_SUITE
NAME_CLIENT_TYPE
NAME_GOODS_CATEGORY
NAME_PORTFOLIO
NAME_PORTFOLIO_TYPE
CHANNEL_TYPE
SELLERPLACE_AREA -------------------
NAME_SELLER_INDUSTRY
NAME_YIELD_GROUP
PRODUCT_COMBINATION
NFLAG_INSURED_ON_APPROVAL --------